<a href="https://colab.research.google.com/github/bdemirc1/chemBERT/blob/main/ChemBERTa_SMILES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# In order to use deepchem within google colab

!curl -Lo conda_installer.py https://raw.githubusercontent.com/deepchem/deepchem/master/scripts/colab_install.py
import conda_installer
conda_installer.install()
!/root/miniconda/bin/conda info -e


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3457  100  3457    0     0  12204      0 --:--:-- --:--:-- --:--:-- 12215


add /root/miniconda/lib/python3.10/site-packages to PYTHONPATH
INFO:conda_installer:add /root/miniconda/lib/python3.10/site-packages to PYTHONPATH
python version: 3.10.12
INFO:conda_installer:python version: 3.10.12
fetching installer from https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
INFO:conda_installer:fetching installer from https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
done
INFO:conda_installer:done
installing miniconda to /root/miniconda
INFO:conda_installer:installing miniconda to /root/miniconda
done
INFO:conda_installer:done
installing openmm, pdbfixer
INFO:conda_installer:installing openmm, pdbfixer
added conda-forge to channels
INFO:conda_installer:added conda-forge to channels
done
INFO:conda_installer:done
conda packages installation finished!
INFO:conda_installer:conda packages installation finished!


# conda environments:
#
base                     /root/miniconda



In [2]:
# Deepchem library consists of necessary modules
!pip install --pre deepchem
import deepchem
deepchem.__version__

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.0/906.0 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 42.2 MB/s eta 0:00:00


'2.7.2.dev'

In [3]:
# Nvidia Apex for dealing with 16-bit training-less memory but mixed precision training
# Apex alleviates these problems.
!git clone https://github.com/NVIDIA/apex
%cd apex
!pwd


Cloning into 'apex'...
remote: Enumerating objects: 11486, done.
remote: Counting objects: 100% (3554/3554), done.
remote: Compressing objects: 100% (481/481), done.
remote: Total 11486 (delta 3235), reused 3176 (delta 3070), pack-reused 7932
Receiving objects: 100% (11486/11486), 15.42 MiB | 12.65 MiB/s, done.
Resolving deltas: 100% (8064/8064), done.
/content/apex
/content/apex


In [4]:
!pip install -v --disable-pip-version-check --no-cache-dir --global-option="bdist_wheel" --global-option="--cpp_ext" --global-option="--cuda_ext" ./

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
DEPRECATION: --build-option and --global-option are deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use --config-settings. Discussion can be found at https://github.com/pypa/pip/issues/11859
Processing /content/apex
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 11.7 MB/s eta 0:00:00
    Getting requirements to build wheel: started
    Running command Getting requirements to build wheel
    running egg_info
    writing setuptools.egg-info/PKG-INFO
    writing dependency_links to setuptools.egg-info/dependency_links.txt
    writing entry points to setuptools.egg-info/entry_points.txt
    writing requirements to setuptools.egg-info/requires.txt
    writing top-level names to setuptools.egg-info/top_level.txt
    readi

In [5]:
# Required for simple transformer classification model - trained using f16 - true: 16 bit
from apex import amp

In [6]:
%cd ..
!git clone https://github.com/seyonechithrananda/bert-loves-chemistry.git

/content
Cloning into 'bert-loves-chemistry'...
remote: Enumerating objects: 1566, done.
remote: Counting objects: 100% (202/202), done.
remote: Compressing objects: 100% (111/111), done.
remote: Total 1566 (delta 96), reused 91 (delta 91), pack-reused 1364
Receiving objects: 100% (1566/1566), 55.35 MiB | 16.95 MiB/s, done.
Resolving deltas: 100% (1000/1000), done.
Updating files: 100% (122/122), done.


In [7]:
!pip install transformers
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.8/250.8 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.2/249.2 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 65.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 12.4 MB

In [9]:
!nvidia-smi

Wed Nov 22 19:13:17 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
%cd /content/bert-loves-chemistry

/content/bert-loves-chemistry


In [10]:
!pwd

/content/bert-loves-chemistry


In [11]:
import os

import numpy as np
import pandas as pd

from deepchem.molnet import load_bbbp, load_clearance, load_clintox, load_delaney, load_hiv, load_qm7, load_tox21
from rdkit import Chem


# import MolNet dataloder from bert-loves-chemistry
from chemberta.utils.molnet_dataloader import load_molnet_dataset, write_molnet_dataset_for_chemprop


In [12]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")  # GPU available
else:
    device = torch.device("cpu")  # No GPU, fall back to CPU


In [13]:
df_density = pd.read_csv("../10k_dft_density_data.csv")
df_density.dtypes
df_density.info

<bound method DataFrame.info of                                                   smiles   density
0                           CC(C)OC(=O)c1ccc(cc1)N(=O)=O  1.367633
1                 CCc1ccc(cc1)C=C(COc1ccccc1C=NO)C(=O)OC  1.292618
2                              CC(C)(C)c1ccc(O)c(C=NO)c1  1.227993
3                      Cc1ccccc1NN=C(C)c1cccc(c1)N(=O)=O  1.316755
4      CN(C)c1ccc(cc1)C(=C(C#N)C#N)C(c1ccc(cc1)N(=O)=...  1.406207
...                                                  ...       ...
10201         CON1C(=C(OC(C)=O)C(=C1C=Cc1ccccc1)C(C)=O)C  1.299822
10202                        CON=C(C)C(=O)C=CC=Cc1ccccc1  1.220360
10203                          CON=C(C)C(=O)C=CC1=CC=CN1  1.261450
10204      CN(c1ccccn1)c1c(cc(cc1N(=O)=O)N(=O)=O)N(=O)=O  1.579706
10205  Cc1c(nc2CC(C)(C)CC(=O)c2c1c1ccc(cc1)N(=O)=O)c1...  1.333787

[10206 rows x 2 columns]>

In [14]:
new_column_names = {
    "smiles": "text",
    "density": "labels",
}
df_density.rename(columns=new_column_names, inplace=True)
df_density

,text,labels
0,CC(C)OC(=O)c1ccc(cc1)N(=O)=O,1.367633
1,CCc1ccc(cc1)C=C(COc1ccccc1C=NO)C(=O)OC,1.292618
2,CC(C)(C)c1ccc(O)c(C=NO)c1,1.227993
3,Cc1ccccc1NN=C(C)c1cccc(c1)N(=O)=O,1.316755
4,CN(C)c1ccc(cc1)C(=C(C#N)C#N)C(c1ccc(cc1)N(=O)=...,1.406207
...,...,...
10201,CON1C(=C(OC(C)=O)C(=C1C=Cc1ccccc1)C(C)=O)C,1.299822
10202,CON=C(C)C(=O)C=CC=Cc1ccccc1,1.220360
10203,CON=C(C)C(=O)C=CC1=CC=CN1,1.261450
10204,CN(c1ccccn1)c1c(cc(cc1N(=O)=O)N(=O)=O)N(=O)=O,1.579706


In [15]:
labels_mean = df_density['labels'].mean()
labels_std = df_density['labels'].std()

df_density['labels'] = (df_density['labels'] - labels_mean) / labels_std

print(df_density)

                                                    text    labels
0                           CC(C)OC(=O)c1ccc(cc1)N(=O)=O -0.248155
1                 CCc1ccc(cc1)C=C(COc1ccccc1C=NO)C(=O)OC -0.884582
2                              CC(C)(C)c1ccc(O)c(C=NO)c1 -1.432860
3                      Cc1ccccc1NN=C(C)c1cccc(c1)N(=O)=O -0.679806
4      CN(C)c1ccc(cc1)C(=C(C#N)C#N)C(c1ccc(cc1)N(=O)=...  0.079108
...                                                  ...       ...
10201         CON1C(=C(OC(C)=O)C(=C1C=Cc1ccccc1)C(C)=O)C -0.823465
10202                        CON=C(C)C(=O)C=CC=Cc1ccccc1 -1.497615
10203                          CON=C(C)C(=O)C=CC1=CC=CN1 -1.149008
10204      CN(c1ccccn1)c1c(cc(cc1N(=O)=O)N(=O)=O)N(=O)=O  1.551068
10205  Cc1c(nc2CC(C)(C)CC(=O)c2c1c1ccc(cc1)N(=O)=O)c1... -0.535304

[10206 rows x 2 columns]


In [16]:
from sklearn.model_selection import train_test_split

train_density_df, temp_density_df = train_test_split(df_density, test_size=0.3, random_state=42)
validation_density_df, test_density_df = train_test_split(temp_density_df, test_size=0.5, random_state=42)

In [17]:
# Fetch Smiles tokenizer - vocab per line
!wget https://deepchemdata.s3-us-west-1.amazonaws.com/datasets/vocab.txt

--2023-11-22 19:19:48--  https://deepchemdata.s3-us-west-1.amazonaws.com/datasets/vocab.txt
Resolving deepchemdata.s3-us-west-1.amazonaws.com (deepchemdata.s3-us-west-1.amazonaws.com)... 52.219.194.138, 52.219.112.65, 52.219.193.162, ...
Connecting to deepchemdata.s3-us-west-1.amazonaws.com (deepchemdata.s3-us-west-1.amazonaws.com)|52.219.194.138|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3524 (3.4K) [text/plain]
Saving to: ‘vocab.txt’

vocab.txt           100%[===================>]   3.44K  --.-KB/s    in 0s      

2023-11-22 19:19:48 (176 MB/s) - ‘vocab.txt’ saved [3524/3524]



In [20]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import logging
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [21]:
model_args = ClassificationArgs()
model_args.num_train_epochs = 1
model_args.regression = True
model_args.evaluate_each_epoch = True
model_args.evaluate_during_training_verbose = True
model_args.no_save = True
model_args.num_train_epochs = 10
model_args.wandb_project = 'chemBert10k-smiles'
model_args.overwrite_output_dir = True

In [22]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [26]:
model = ClassificationModel(
    "roberta",
    'seyonec/SMILES_tokenized_PubChem_shard00_160k',
    num_labels=1,
    args=model_args
)

config.json:   0%|          | 0.00/515 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/336M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at seyonec/SMILES_tokenized_PubChem_shard00_160k and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/327 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/8.14k [00:00<?, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [27]:
print(model.tokenizer)

RobertaTokenizerFast(name_or_path='seyonec/SMILES_tokenized_PubChem_shard00_160k', vocab_size=591, model_max_length=514, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	11: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	12: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	13: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	14: AddedToken("[MASK]", rstrip=False, lstrip=True, single_word=False, normalized=False, special=True),
	591: AddedToken("<s>", rstrip=False, lstrip=False, single_word=Fals

In [28]:
# Train the model
model.train_model(train_density_df)

  0%|          | 0/7144 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

wandb: Currently logged in as: bdemirc1 (bingedu). Use `wandb login --relogin` to force relogin


Running Epoch 0 of 10:   0%|          | 0/893 [00:00<?, ?it/s]

Running Epoch 1 of 10:   0%|          | 0/893 [00:00<?, ?it/s]

...

Running Epoch 6 of 10:   0%|          | 0/893 [00:00<?, ?it/s]

Running Epoch 7 of 10:   0%|          | 0/893 [00:00<?, ?it/s]

Running Epoch 8 of 10:   0%|          | 0/893 [00:00<?, ?it/s]

Running Epoch 9 of 10:   0%|          | 0/893 [00:00<?, ?it/s]

(8930, 0.04224287969133768)

In [29]:
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(validation_density_df)

  0%|          | 0/1531 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/192 [00:00<?, ?it/s]

In [30]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Convert text input data to list of strings
text_inputs = test_density_df['text'].tolist()

# Predict target values
predictions, model_outputs = model.predict(text_inputs)
print(predictions)

# Calculate MAE and MSE
mae = mean_absolute_error(test_density_df['labels'], predictions)
mse = mean_squared_error(test_density_df['labels'], predictions)
rmse = np.sqrt(mse)

# Print the results
print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)
print("Root Mean Squared Error normalized:", rmse)
# print(labels_std)
print("Root Mean Squared Error original:", rmse * labels_std)

  0%|          | 0/1531 [00:00<?, ?it/s]

  0%|          | 0/192 [00:00<?, ?it/s]

[-0.65576172 -0.73193359  1.55957031 ...  0.20202637 -0.84619141
 -0.72460938]
Mean Absolute Error: 0.06335083220323544
Mean Squared Error: 0.007019719779228375
Root Mean Squared Error normalized: 0.08378376799373716
0.11786908548881467
Root Mean Squared Error original: 0.009875516112228819


In [31]:
# HoF Data
df_hof = pd.read_csv("../10k_dft_hof_data.csv")
df_hof.dtypes
df_hof.info

<bound method DataFrame.info of                                                   smiles       hof_s
0                           CC(C)OC(=O)c1ccc(cc1)N(=O)=O -109.985346
1                 CCc1ccc(cc1)C=C(COc1ccccc1C=NO)C(=O)OC -100.962611
2                              CC(C)(C)c1ccc(O)c(C=NO)c1  -55.199618
3                      Cc1ccccc1NN=C(C)c1cccc(c1)N(=O)=O   34.675794
4      CN(C)c1ccc(cc1)C(=C(C#N)C#N)C(c1ccc(cc1)N(=O)=...   49.411500
...                                                  ...         ...
10201         CON1C(=C(OC(C)=O)C(=C1C=Cc1ccccc1)C(C)=O)C  -99.026131
10202                        CON=C(C)C(=O)C=CC=Cc1ccccc1  -12.290865
10203                          CON=C(C)C(=O)C=CC1=CC=CN1  -12.217791
10204      CN(c1ccccn1)c1c(cc(cc1N(=O)=O)N(=O)=O)N(=O)=O   29.753252
10205  Cc1c(nc2CC(C)(C)CC(=O)c2c1c1ccc(cc1)N(=O)=O)c1...  -33.978039

[10206 rows x 2 columns]>

In [32]:
new_column_names = {
    "smiles": "text",
    "hof_s": "labels",
}
df_hof.rename(columns=new_column_names, inplace=True)
df_hof

,text,labels
0,CC(C)OC(=O)c1ccc(cc1)N(=O)=O,-109.985346
1,CCc1ccc(cc1)C=C(COc1ccccc1C=NO)C(=O)OC,-100.962611
2,CC(C)(C)c1ccc(O)c(C=NO)c1,-55.199618
3,Cc1ccccc1NN=C(C)c1cccc(c1)N(=O)=O,34.675794
4,CN(C)c1ccc(cc1)C(=C(C#N)C#N)C(c1ccc(cc1)N(=O)=...,49.411500
...,...,...
10201,CON1C(=C(OC(C)=O)C(=C1C=Cc1ccccc1)C(C)=O)C,-99.026131
10202,CON=C(C)C(=O)C=CC=Cc1ccccc1,-12.290865
10203,CON=C(C)C(=O)C=CC1=CC=CN1,-12.217791
10204,CN(c1ccccn1)c1c(cc(cc1N(=O)=O)N(=O)=O)N(=O)=O,29.753252


In [33]:
labels_mean = df_hof['labels'].mean()
labels_std = df_hof['labels'].std()

df_hof['labels'] = (df_hof['labels'] - labels_mean) / labels_std

print(df_hof)

                                                    text    labels
0                           CC(C)OC(=O)c1ccc(cc1)N(=O)=O -0.396132
1                 CCc1ccc(cc1)C=C(COc1ccccc1C=NO)C(=O)OC -0.312696
2                              CC(C)(C)c1ccc(O)c(C=NO)c1  0.110488
3                      Cc1ccccc1NN=C(C)c1cccc(c1)N(=O)=O  0.941594
4      CN(C)c1ccc(cc1)C(=C(C#N)C#N)C(c1ccc(cc1)N(=O)=...  1.077860
...                                                  ...       ...
10201         CON1C(=C(OC(C)=O)C(=C1C=Cc1ccccc1)C(C)=O)C -0.294789
10202                        CON=C(C)C(=O)C=CC=Cc1ccccc1  0.507279
10203                          CON=C(C)C(=O)C=CC1=CC=CN1  0.507955
10204      CN(c1ccccn1)c1c(cc(cc1N(=O)=O)N(=O)=O)N(=O)=O  0.896074
10205  Cc1c(nc2CC(C)(C)CC(=O)c2c1c1ccc(cc1)N(=O)=O)c1...  0.306731

[10206 rows x 2 columns]


In [34]:
train_hof_df, temp_hof_df = train_test_split(df_hof, test_size=0.3, random_state=42)
validation_hof_df, test_hof_df = train_test_split(temp_hof_df, test_size=0.5, random_state=42)

In [35]:
model_args = ClassificationArgs()
model_args.num_train_epochs = 1
model_args.regression = True
model_args.evaluate_each_epoch = True
model_args.evaluate_during_training_verbose = True
model_args.no_save = True
model_args.num_train_epochs = 10
model_args.wandb_project = 'chemBert10k-smiles'
model_args.overwrite_output_dir = True

In [37]:
model = ClassificationModel(
    "roberta",
    'seyonec/SMILES_tokenized_PubChem_shard00_160k',
    num_labels=1,
    args=model_args
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at seyonec/SMILES_tokenized_PubChem_shard00_160k and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [38]:
# Train the model
model.train_model(train_hof_df)

  0%|          | 0/7144 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Training loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
lr,▂▅████▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
Training loss,0.00174
global_step,8900
lr,0.0


Running Epoch 0 of 10:   0%|          | 0/893 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


...

Running Epoch 6 of 10:   0%|          | 0/893 [00:00<?, ?it/s]

Running Epoch 7 of 10:   0%|          | 0/893 [00:00<?, ?it/s]

Running Epoch 8 of 10:   0%|          | 0/893 [00:00<?, ?it/s]

Running Epoch 9 of 10:   0%|          | 0/893 [00:00<?, ?it/s]

(8930, 0.05417177194599344)

In [39]:
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(validation_hof_df)

  0%|          | 0/1531 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/192 [00:00<?, ?it/s]

In [40]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
text_inputs = test_hof_df['text'].tolist()

predictions, model_outputs = model.predict(text_inputs)
print(predictions)

# Calculate MAE and MSE
mae = mean_absolute_error(test_hof_df['labels'], predictions)
mse = mean_squared_error(test_hof_df['labels'], predictions)
rmse = np.sqrt(mse)

# Print the results
print("HoF Mean Absolute Error:", mae)
print("HoF Mean Squared Error:", mse * (labels_std**2))
print("HoF Root Mean Squared Error normalized:", rmse)
print("HoF Root Mean Squared Error original:", rmse * labels_std)

  0%|          | 0/1531 [00:00<?, ?it/s]

  0%|          | 0/192 [00:00<?, ?it/s]

[-1.26660156  1.11914062  0.31298828 ...  0.79785156  0.671875
  0.95117188]
HoF Mean Absolute Error: 0.1102797494305863
HoF Mean Squared Error: 253.23094756900565
HoF Root Mean Squared Error normalized: 0.14715462255656211
HoF Root Mean Squared Error original: 15.913231839227556


Building the network

In [ ]:
from transformers import AutoModel, AutoTokenizer, AdamW
import torch
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight

# Load the model and tokenizer
model_name = "seyonec/ChemBERTA_PubChem1M_shard00_155k"
model = AutoModel.from_pretrained(model_name, num_labels=1)
model.to(device)

tokenizer = AutoTokenizer.from_pretrained(model_name)
print(tokenizer)

# Define a custom dataset
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(text, truncation=True, padding='max_length', max_length=self.max_length, return_tensors='pt')
        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': torch.tensor(label, dtype=torch.long)
        }


(…)1M_shard00_155k/resolve/main/config.json:   0%|          | 0.00/515 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/336M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at seyonec/ChemBERTA_PubChem1M_shard00_155k and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


(…)_155k/resolve/main/tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

(…)m1M_shard00_155k/resolve/main/vocab.json:   0%|          | 0.00/52.6k [00:00<?, ?B/s]

(…)m1M_shard00_155k/resolve/main/merges.txt:   0%|          | 0.00/28.7k [00:00<?, ?B/s]

(…)55k/resolve/main/special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

RobertaTokenizerFast(name_or_path='seyonec/ChemBERTA_PubChem1M_shard00_155k', vocab_size=3007, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	4: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True, special=True),
}


In [ ]:
# Spliting the data into training, validation, and test sets
train_texts, train_labels = train_density_df['text'].tolist(), train_density_df['labels'].tolist()
valid_texts, valid_labels = validation_density_df['text'].tolist(), validation_density_df['labels'].tolist()
test_texts, test_labels = test_density_df['text'].tolist(), test_density_df['labels'].tolist()

# Creating datasets
train_dataset = CustomDataset(train_texts, train_labels, tokenizer)
valid_dataset = CustomDataset(valid_texts, valid_labels, tokenizer)
test_dataset = CustomDataset(test_texts, test_labels, tokenizer)

# Creainge data loaders
batch_size = 32
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, pin_memory=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, pin_memory=True)

In [ ]:
# Define the optimizer (AdamW)
optimizer = AdamW(model.parameters(), lr=1e-3)

#class_labels = [0,1]
#class_counts = [1183 - 1105, 1105]
#class_weights = compute_class_weight('balanced', classes=class_labels, y=train_labels) #  n_samples / (n_classes * np.bincount(y))

print(class_weights)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float)
class_weights_tensor = class_weights_tensor.to(device)

# Define the Binary Cross-Entropy (BCE) loss
criterion = torch.nn.BCEWithLogitsLoss()

# Training loop
num_epochs = 1 #

for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        input_ids = batch['input_ids']
        print(input_ids.shape)
        input_ids = input_ids.to(device)
        attention_mask = batch['attention_mask']
        attention_mask = attention_mask.to(device)
        labels = batch['labels']
        labels = labels.to(device)
        print(labels.shape)
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask).logits
        outputs = outputs.squeeze()
        print(outputs.shape)
        outputs = outputs.to(device)
        loss = criterion(outputs, labels.float()).to(device)
        loss.backward()
        optimizer.step()

    # Initialize lists to store true labels, predicted labels (for validation), or predictions (for test data)
    true_labels_val = []
    predicted_labels_val = []
    predictions_test = []

    # Validation
    model.eval()
    with torch.no_grad():
        total_loss = 0.0
        num_samples = 0
        for batch in valid_dataloader:
            input_ids = batch['input_ids']
            input_ids = input_ids.to(device)
            attention_mask = batch['attention_mask']
            attention_mask = attention_mask.to(device)
            labels = batch['labels'].unsqueeze(1)
            labels = labels.to(device)
            outputs = model(input_ids, attention_mask=attention_mask).logits
            outputs = outputs.to(device)
            loss = criterion(outputs, labels.float())
            total_loss += loss.item()
            num_samples += len(input_ids)

            # Convert to numpy arrays and store true and predicted labels (for validation)
            true_labels_val.extend(labels.cpu().numpy())
            predicted_labels_val.extend(outputs.cpu().numpy())

        average_loss = total_loss / num_samples
        print(f'Epoch [{epoch + 1}/{num_epochs}], Validation Loss: {average_loss:.4f}')

        # Calculate binary classification metrics (for validation)
        true_labels_val = [1 if label > 0.5 else 0 for label in true_labels_val]
        predicted_labels_val = [1 if label > 0.5 else 0 for label in predicted_labels_val]

        accuracy = accuracy_score(true_labels_val, predicted_labels_val)
        precision = precision_score(true_labels_val, predicted_labels_val)
        recall = recall_score(true_labels_val, predicted_labels_val)
        f1 = f1_score(true_labels_val, predicted_labels_val)
        conf_matrix = confusion_matrix(true_labels_val, predicted_labels_val)

        print(f'Validation Accuracy: {accuracy:.4f}')
        print(f'Validation Precision: {precision:.4f}')
        print(f'Validation Recall: {recall:.4f}')
        print(f'Validation F1 Score: {f1:.4f}')
        print('Validation Confusion Matrix:')
        print(conf_matrix)

# Inference on the test data
''' model.eval()
predictions = []
with torch.no_grad():
    for batch in test_dataloader:
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        outputs = model(input_ids, attention_mask=attention_mask).logits
        predictions.extend(outputs.tolist()) '''

#print("Predictions:", predictions)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


[7.49367089 0.53574661]
torch.Size([32, 128])
torch.Size([32])
torch.Size([32])
torch.Size([32, 128])
torch.Size([32])
torch.Size([32])
torch.Size([32, 128])
torch.Size([32])
torch.Size([32])
torch.Size([32, 128])
torch.Size([32])
torch.Size([32])
torch.Size([32, 128])
torch.Size([32])
torch.Size([32])
torch.Size([32, 128])
torch.Size([32])
torch.Size([32])
torch.Size([32, 128])
torch.Size([32])
torch.Size([32])
torch.Size([32, 128])
torch.Size([32])
torch.Size([32])
torch.Size([32, 128])
torch.Size([32])
torch.Size([32])
torch.Size([32, 128])
torch.Size([32])
torch.Size([32])
torch.Size([32, 128])
torch.Size([32])
torch.Size([32])
torch.Size([32, 128])
torch.Size([32])
torch.Size([32])
torch.Size([32, 128])
torch.Size([32])
torch.Size([32])
torch.Size([32, 128])
torch.Size([32])
torch.Size([32])
torch.Size([32, 128])
torch.Size([32])
torch.Size([32])
torch.Size([32, 128])
torch.Size([32])
torch.Size([32])
torch.Size([32, 128])
torch.Size([32])
torch.Size([32])
torch.Size([32, 128])
to

" model.eval()\npredictions = []\nwith torch.no_grad():\n    for batch in test_dataloader:\n        input_ids = batch['input_ids']\n        attention_mask = batch['attention_mask']\n        outputs = model(input_ids, attention_mask=attention_mask).logits\n        predictions.extend(outputs.tolist()) "